In [18]:
import mysql.connector
from mysql.connector import Error
import json
import re
import pandas as pd
from datetime import datetime
from DBHelper import CDBHelper
from ClickUpDB import CClickUpDB
import os

In [19]:
list_id = "901600183071"  # Replace with your actual ListID
start_date = "01-08-2024"  # Replace with your desired start date
end_date = "01-09-2024"    # Replace with your desired end date

# Fetch tasks based on the criteria
tasks = CClickUpDB.MSGetTasksByListID(list_id, start_date, end_date)
print(len(tasks))

Successfully connected to MySQL server
Retrieved 54 task(s) from ListID 901600183071 between 01-08-2024 and 01-09-2024.
54


In [30]:
def MSCreateDataframe(lsTasks):
    # Creating a DataFrame from the list of tasks
    df = pd.DataFrame(lsTasks)
    
    # Parsing JSON strings in 'TaskPriority' and 'TaskAssigneesList'
    df['TaskPriority'] = df['TaskPriority'].apply(lambda x: json.loads(x) if x != 'null' else None)
    df['TaskAssigneesList'] = df['TaskAssigneesList'].apply(json.loads)
    df['TaskScore'] = df.apply(CClickUpDB.calculate_task_score, include_toughness=False, axis=1)
    
    df = df[df['EstimatedTime'].apply(CClickUpDB.isEstimatedTimeProvided)]

    # Convert 'TaskStartDate' to datetime for sorting
    # df['TaskStartDate'] = pd.to_datetime(df['TaskStartDate'], format='%d-%m-%Y')

    # Sort tasks by 'TaskStartDate' ascending, then by 'TaskScore' ascending within the same date
    # df = df.sort_values(by=['TaskStartDate', 'TaskScore'])

    # Drop columns except the specified list
    columns_to_keep = ['ListName', 'TaskID', 'TaskSubject', 'TaskStartDate', 'TaskDueDate', 'TaskStatus',
                       'EstimatedTime', 'TaskPriority', 'TaskAssigneesList', 
                       'TaskIsMilestone', 'TaskIntensity', 'TaskScore']
    df = df[columns_to_keep]
    print(df)
    # Add new columns
    # 1. 'EstimatedTaskMin' - sum of hours * 60 + mins
    df['EstimatedTime'] = df['EstimatedTime'].apply(json.loads)
    df['EstimatedTaskMin'] = df['EstimatedTime'].apply(lambda x: (x['hrs'] * 60 + x['mins']) if x else 0)

    # 2. 'AssignTo' - extract username from the first entry in 'TaskAssigneesList'
    df['AssignTo'] = df['TaskAssigneesList'].apply(lambda x: x[0]['username'] if x and len(x) > 0 else None)

    # 3. 'TaskPriority' - extract priority
    df['TaskPriority'] = df['TaskPriority'].apply(lambda x: x['priority'] if x else None)

    # 4. 'TaskStatus' - assuming there's a 'TaskStatus' field structured similarly
    df['TaskStatus'] = df['TaskStatus'].apply(json.loads)
    df['TaskStatus'] = df['TaskStatus'].apply(lambda x: x['status'] if isinstance(x, dict) else None)

    # Creating employee-wise task lists
    employee_tasks = {}
    for _, row in df.iterrows():
        assignees = row['TaskAssigneesList']
        if assignees:
            for assignee in assignees:
                username = assignee['username']
                if username not in employee_tasks:
                    employee_tasks[username] = []
                # Convert the row to a dictionary and append to the list
                employee_tasks[username].append(row.to_dict())

    # Directory to save the Excel files
    output_directory = r"Data/"
    os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

    # Save employee-wise task lists to Excel files
    for username, tasks in employee_tasks.items():
        # Create a DataFrame from the list of task dictionaries
        df_employee = pd.DataFrame(tasks)
        # Create a filename based on the username
        filename = f"{username.replace(' ', '_')}_tasks.xlsx"
        # Construct the full path
        file_path = os.path.join(output_directory, filename)
        # Save the DataFrame to an Excel file
        df_employee.to_excel(file_path, index=False)

    print("Employee-wise task details have been saved to the 'Data/' directory.")
    
    # Return the dictionary where keys are employee names and values are lists of task dictionaries
    return employee_tasks


employee_tasks = MSCreateDataframe(tasks)
print(employee_tasks)

   ListName     TaskID                                       TaskSubject  \
3   ERPNext  86cw75tb6                         Mohit - Job Openings Form   
4   ERPNext  86cw75tdf                        Mansi - Leave Request Form   
5   ERPNext  86cw75zvc                             Mohit - Employee Form   
6   ERPNext  86cw76fjj  Mitul - Compare Odoo vs Zoho vs ERPNext Solution   
11  ERPNext  86cw7wkbw                           Restore V14 Data in V15   
15  ERPNext  86cw7wzx2                             Mitul - Review & Test   
16  ERPNext  86cw7wzy0                          Mitul - show demo to AKS   
17  ERPNext  86cw7x1eg                          Mitul - show demo to AKS   
18  ERPNext  86cw7x1eh                             Mitul - Review & Test   
19  ERPNext  86cw7x7xw                      Mansi - Import Projects List   
20  ERPNext  86cw7x7zr     Mitul - Import Customer List & Customize Form   
21  ERPNext  86cw7x82b                        Mohit  - Import Users List   
22  ERPNext 

In [13]:
def processCleanDF(employee_dataframes):
    # Define the maximum hours in a particular day
    max_daily_hours = 8 * 60  # 8 hours converted to minutes

    # Process each employee DataFrame
    for username, df in employee_dataframes.items():
        # Add a new column to indicate if the task fits in a particular day
        df['IsTaskFitInParticularDate'] = 'No'

        # Keep track of remaining time for each day
        remaining_time_in_minutes = max_daily_hours

        # Iterate through each task row
        for index, row in df.iterrows():
            print("row",isinstance(row['EstimatedTime'],str))
            # Get estimated hours and minutes
            
            estimated_hours = row['EstimatedTime'].get('hrs', 0)
            estimated_minutes = row['EstimatedTime'].get('mins', 0)
            total_task_minutes = estimated_hours * 60 + estimated_minutes

            # Check if the task fits within the remaining time for the day
            if total_task_minutes <= remaining_time_in_minutes:
                # Mark as "Yes" if it fits
                df.at[index, 'IsTaskFitInParticularDate'] = 'Yes'
                # Subtract the time from the remaining time for the day
                remaining_time_in_minutes -= total_task_minutes
            else:
                # If it doesn't fit, reset the remaining time for the next day
                remaining_time_in_minutes = max_daily_hours - total_task_minutes
                # Mark this task as fitting in the new day
                df.at[index, 'IsTaskFitInParticularDate'] = 'Yes'

    # Optionally, return the modified DataFrames if needed
    return employee_dataframes
        

processCleanDF(tasks)

[{'ListName': 'ERPNext', 'ListID': '901600183071', 'FolderName': 'REAL Office', 'SpaceID': '90020386592', 'TaskID': '86cw73j5j', 'TaskSubject': 'Mitul - ERPNext Install & Setup V15', 'TaskStartDate': '13-08-2024', 'TaskDueDate': '21-08-2024', 'ParentTaskID': '86cw76fxd', 'EstimatedTime': '{"hrs": 0, "mins": 0, "time_estimate": null}', 'TaskPriority': 'null', 'TaskStatus': '{"id": "subcat901600183071_subcat901600182452_subcat900202016677_subcat900202016557_subcat900202016465_subcat900201647299_subcat900201617233_subcat900201614169_subcat900201614016_subcat900201613908_subcat900201524923_subcat900900756278_subcat900900756275_subcat900900756272_subcat900600588774_sc900600504232_NTf0x6rB", "type": "closed", "color": "#008844", "status": "delievered", "orderindex": 4}', 'AssignByPersonDetails': '{"id": 67390920, "color": "", "email": "mitul@riveredgeanalytics.com", "username": "Mitul Solanki", "profilePicture": "https://attachments.clickup.com/profilePictures/67390920_BGd.jpg"}', 'TaskAssig